In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

In [2]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/train/*.feather"))
test1_filenames = building1_filenames - training_filenames

In [5]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [6]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)
training_data.columns
#training_data.head()

orientation_acc = training_data[["iphoneAccX", "iphoneAccY", "iphoneAccZ"]]
#orientation_acc.drop([0], inplace=True)
acc_np = orientation_acc.to_numpy()
orientation_gyro = training_data[[ "iphoneGyroX", "iphoneGyroY", "iphoneGyroZ"]]
#orientation_gyro.drop([0], inplace=True)
gyro_np = orientation_gyro.to_numpy()

In [7]:
acc_np[:100,:].shape

(100, 3)

In [8]:
position_target = training_data[["processedPosX", "processedPosY" ]]#, "processedPosZ"]] 

position_diff_target = position_target.diff()
position_diff_target.drop([0], inplace=True)
position_diff_target.head()

pos_np = position_target.to_numpy()
len(pos_np)

2087464

In [9]:
def generate_training_samples(acc_, gyro_, pos_, batch_size = 64):    
    while True:
        xa_batch = np.zeros((batch_size,100,3))
        xg_batch = np.zeros((batch_size,100,3))
        y_batch = np.zeros((batch_size,2))
        j = 0
        for i in range(len(pos_)):
            xa_batch[j,:,:] = acc_[i, :]
            xg_batch[j,:,:] = gyro_[i,:]
            y_batch[j,:] = pos_[i,:]
            j += 1
            if j >= batch_size:
                j = 0              
                yield([xa_batch,xg_batch],y_batch)
            #---------------------------------------------------------------------------------------

In [10]:
##gyro = ['iphoneGyroX', 'iphoneGyroY', 'iphoneGyroZ']
##acc = ["iphoneAccX", "iphoneAccY", "iphoneAccZ"]
##pos = ["processedPosX", "processedPosY"]#, "processedPosZ"]
##
##in_dataset_2 = (
##    tf.data.Dataset.from_tensor_slices(
##        (
##            tf.cast(orientation_gyro[gyro].values, tf.float32)
##        )
##    )
##)
##
##in_dataset_1 = (
##    tf.data.Dataset.from_tensor_slices(
##        (
##            
##            tf.cast(orientation_acc[acc].values, tf.float32)
##        )
##    )
##)
##
##out_dataset = (tf.data.Dataset.from_tensor_slices((tf.cast(position_diff_target[pos].values, tf.int32))))
##
##
### Hack -> open TF issue: https://github.com/tensorflow/tensorflow/issues/44046
##in_dataset_1 =in_dataset_1.window(100,1,1)
##in_dataset_1 = in_dataset_1.flat_map(lambda x:x).batch(100).batch(64) #flat map but do nothing to values, then batch twice to get what model.fit expects from windowed datasets
##in_dataset_2 =in_dataset_2.window(100,1,1)
##in_dataset_2 = in_dataset_2.flat_map(lambda x:x).batch(100).batch(64) #flat map but do nothing to values, then batch twice to get what model.fit expects from windowed datasets
##
##out_dataset = out_dataset.window(100,1,1)
##out_dataset = out_dataset.flat_map(lambda x:x).batch(100).batch(64)
##in_dataset =  tf.data.Dataset.zip(((in_dataset_1, in_dataset_2), out_dataset))
###training_dataset = tf.data.Dataset.zip((in_dataset, out_dataset))
###seq_training_dataset = training_dataset.window(100, shift=1)
###windows = next(iter(seq_training_dataset))
#ds = training_dataset

In [11]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_PostNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_vloss_{val_loss:.4f}.hdf5')
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [12]:
pos_learning_rate = 0.001

In [13]:

def build_model():
    # Building Network
    #1. Define inputs
    Acc_input = Input(shape=(100,3) , name = 'Acc_input')    
    Gyro_input = Input(shape=(100,3) , name = 'Gyro_input')
    
    MergedLayer = Concatenate()([Acc_input , Gyro_input])
    #
    ##MergedLayer = Input(shape=(9) , name = 'imu_input')Bidirectional(
    #2. LSTM
    LSTM1 = (Bidirectional(LSTM(100, return_sequences=True)))(MergedLayer) # , return_sequences=True
    LSTM2 = (Bidirectional(LSTM(100)))(LSTM1)
    

    
    #4. Fully-Connected (x)
    Dense3 = Dense(units=100, activation='tanh')(LSTM2)
    Dense4 = Dense(units=20, activation='tanh')(Dense3)
    x_output = Dense(units=2, activation='linear')(Dense4) # Theta Outputs
    
    #5. Define and compile The model
    Network = Model([Acc_input,Gyro_input], [x_output]) # , sigma_output])  #  [Acc_input,Gyro_input, Mag_input]
    Network.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=pos_learning_rate) ,metrics=['mape']) #loss=OreintLoss()
    return Network
    

In [14]:
#network_builder = PosNet()
position_network = build_model()
position_network.summary()

2021-10-24 11:33:31.353900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 11:33:31.359176: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 11:33:31.359439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-24 11:33:31.359992: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Acc_input (InputLayer)          [(None, 100, 3)]     0                                            
__________________________________________________________________________________________________
Gyro_input (InputLayer)         [(None, 100, 3)]     0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 100, 6)       0           Acc_input[0][0]                  
                                                                 Gyro_input[0][0]                 
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 200)     85600       concatenate[0][0]            

In [15]:
Nepochs = 20
train_dataset = tf.data.Dataset.from_tensor_slices((acc_np, gyro_np)).window(100,1,1)
out_data = tf.data.Dataset.from_tensor_slices((pos_np)).window(100,1,1)
dataset =  tf.data.Dataset.zip((train_dataset, out_data))


history = position_network.fit(
    generate_training_samples(acc_np, gyro_np, pos_np),
    epochs=Nepochs,
    initial_epoch=initial_epoch
    #callbacks = [check_point_callback]
)

#     [orientation_acc, 
#    orientation_gyro],
#    position_diff_target,

2021-10-24 11:33:32.719226: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2021-10-24 11:33:35.834440: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


 113340/Unknown - 2113s 19ms/step - loss: 46.9327 - mape: 11715.5869

KeyboardInterrupt: 